#Common functions for all models

### Installing Packages & Importing Libraries

In [ ]:
!pip install pyreadstat

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import transformers

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from transformers import AutoTokenizer,AutoModelForCausalLM
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr

### Functions for WVS Dataset

In [3]:
#Loading the WVS file and storing it in a dataframe format

def get_wvs_df():
    wvs_df = pd.read_csv('sample_data/WVS_Moral.csv') #WVS_Moral is a subset of the full data for just the moral questions
    wvs_df_country_names = pd.read_csv('sample_data/Country_Codes_Names.csv')
    wvs_df = wvs_df.set_index('B_COUNTRY').join(wvs_df_country_names.set_index('B_COUNTRY'), how='left')
    return wvs_df

In [25]:
#Listing all the countries we are gonna use in our study from the WVS dataset

COUNTRIES_WVS_W7_ALL = [
    'Andorra', 'Argentina', 'Armenia', 'Australia', 'Bangladesh', 'Bolivia', 'Brazil', 'Canada',
    'Chile', 'China', 'Colombia', 'Cyprus', 'Ecuador', 'Egypt', 'Ethiopia', 'Germany', 'Greece',
    'Guatemala', 'Indonesia', 'Iran', 'Iraq', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya',
    'Kyrgyzstan', 'Lebanon', 'Libya', 'Malaysia', 'Maldives', 'Mexico', 'Mongolia', 'Morocco',
    'Myanmar', 'Netherlands', 'New Zealand', 'Nicaragua', 'Nigeria', 'Pakistan', 'Peru',
    'Philippines', 'Romania', 'Russia', 'Singapore', 'South Korea', 'Taiwan ROC', 'Tajikistan',
    'Thailand', 'Tunisia', 'Turkey', 'Ukraine', 'United States', 'Venezuela',
    'Vietnam', 'Zimbabwe'
]

In [5]:
#Creating the questions for the WVS dataset

W7_QUESTIONS = ['Q'+str(i) for i in range(177, 196)]

W7_QUESTIONS_TEXT = ['claiming government benefits to which you are not entitled',
                     'avoiding a fare on public transport',
                     'stealing property',
                     'cheating on taxes',
                     'someone accepting a bribe in the course of their duties',
                     'homosexuality',
                     'prostitution',
                     'abortion',
                     'divorce',
                     'sex before marriage',
                     'suicide',
                     'euthanasia',
                     'for a man to beat his wife',
                     'parents beating children',
                     'violence against other people',
                     'terrorism as a political, ideological or religious mean',
                     'having casual sex',
                     'political violence',
                     'death penalty']

QUESTION_WAVES_WVS = {7: (W7_QUESTIONS, W7_QUESTIONS_TEXT)}

In [6]:
#Scaling the survey results between -1 and 1 and calculating the mean rating for a every survey question and culture

MINUS = 5.5
DIVIDE = 4.5

def get_wvs_ratings(wvs_df, culture, q):
    df = wvs_df[['Country_Names', q]]
    df = df.loc[df['Country_Names'] == culture]
    if len(df) == 0:
        return None

    ratings = df.loc[df[q] > 0][q]
    if ratings.empty:
        ratings = 0
    else:
      ratings = ((ratings - MINUS) / DIVIDE).mean()
    return ratings

In [7]:
#Generating the WVS prompts

def wvs_gpt2_prompts_ratings_multiple_tokens(wvs_df:pd.DataFrame, culture ='', prompt_mode ='in', wave = 7, ):
    prompts = {}
    culture_prompt = ''
    if len(culture) > 0:
        if prompt_mode == 'in':
            print("The token used is IN")
            culture_prompt = f'In {culture} '
        elif prompt_mode == 'people':
            print("The token used is PEOPLE")
            culture_prompt = f'People in {culture} believe '

    questions, questions_text = QUESTION_WAVES_WVS[wave]

    for q,q_text in zip(questions, questions_text):
        rating_answers = get_wvs_ratings(wvs_df, culture, q) #getting the original ratings

        prompts[q_text] = []
        for (prompt_head_moral, prompt_head_nonmoral) in TOKEN_PAIRS:
            prompt_moral = f'{culture_prompt}{q_text} is {prompt_head_moral}.'
            prompt_nonmoral = f'{culture_prompt}{q_text} is {prompt_head_nonmoral}.'

            prompts[q_text].append((prompt_moral, prompt_nonmoral,rating_answers))

    return prompts

### Functions for PEW Dataset

In [8]:
#Loading the PEW file, pre-processing it and storing it in a dataframe

def get_pew_df():
    pew_data_original = pd.read_spss('sample_data/Pew Research Global Attitudes Project Spring 2013 Dataset for web.sav')

    filtered_columns = pew_data_original.filter(regex='^Q84[A-H]|COUNTRY').copy()

    filtered_columns.rename(columns={'COUNTRY': 'Country_Names'}, inplace=True)

    replace_map = {
        'Morally acceptable': 1,
        'Not a moral issue': 0,
        'Morally unacceptable': -1,
        'Depends on situation (Volunteered)': 0,
        'Refused': 0,
        "Don't know": 0
    }

    filtered_columns.replace(replace_map, inplace=True)

    for col in filtered_columns.columns[1:]:
        filtered_columns[col] = pd.to_numeric(filtered_columns[col], errors='coerce')

    return filtered_columns

In [9]:
#Listing all the countries we are gonna use in our study from the PEW dataset

COUNTRIES_PEW_ALL = [
    'United States', 'Czech Republic', 'South Korea', 'Canada', 'France', 'Germany',
    'Spain', 'Mexico', 'Chile', 'Australia', 'Russia', 'Britain', 'Turkey', 'Greece',
    'Egypt', 'Poland', 'Senegal', 'Italy', 'Brazil', 'Lebanon', 'Nigeria', 'Japan',
    'Malaysia', 'Kenya', 'Indonesia', 'Uganda', 'Jordan', 'Argentina', 'Philippines',
    'Tunisia', 'China', 'Pakistan', 'Ghana', 'South Africa', 'Palestinian territories',
    'Israel', 'Bolivia', 'Venezuela', 'El Salvador'
]

In [10]:
#Creating the questions for the PEW dataset

PEW_QUESTIONS = ['Q84' + chr(i) for i in range(ord('A'), ord('H')+1)]

PEW_QUESTIONS_TEXT = ['using contraceptives',
                      'getting a divorce',
                      'having an abortion',
                      'homosexuality',
                      'drinking alcohol',
                      'married people having an affair',
                      'gambling',
                      'sex between unmarried adults']

QUESTION_WAVES_PEW = {13: (PEW_QUESTIONS, PEW_QUESTIONS_TEXT)}

In [11]:
#Calculating the mean rating for a every question and culture

def get_pew_ratings(pew_df, culture, q):
    df = pew_df[['Country_Names', q]]
    df = df.loc[df['Country_Names'] == culture]
    if df.empty:
        print("No data found for culture:", culture)
        return None

    mean_rating = df[q].mean()

    if pd.isna(mean_rating):
        print("Problem: Mean calculation resulted in NaN for culture:", culture)
        return None

    return mean_rating

In [12]:
#Generating the PEW prompts

def pew_gpt2_prompts_ratings_multiple_tokens(pew_df:pd.DataFrame, culture ='', prompt_mode ='in', wave = 13, ):
    prompts = {}
    culture_prompt = ''
    if len(culture) > 0:
        if prompt_mode == 'in':
            culture_prompt = f'In {culture} '
            print("Inside IN")
        elif prompt_mode == 'people':
            culture_prompt = f'People in {culture} believe '
            print("Inside PEOPLE")

    questions, questions_text = QUESTION_WAVES_PEW[wave]

    for q,q_text in zip(questions, questions_text):
        rating_answers = get_pew_ratings(pew_df, culture, q) #getting the original ratings

        prompts[q_text] = []
        for (prompt_head_moral, prompt_head_nonmoral) in TOKEN_PAIRS:
            prompt_moral = f'{culture_prompt}{q_text} is {prompt_head_moral}.'
            prompt_nonmoral = f'{culture_prompt}{q_text} is {prompt_head_nonmoral}.'

            prompts[q_text].append((prompt_moral, prompt_nonmoral,rating_answers))

    return prompts

## Other functions

### normalize_log_prob_diffs() and calculate_correlation()

In [13]:
#Normalizing the log probability differences in a scale between -1 and 1

def normalize_log_prob_diffs(log_prob_diffs):
    min_log_prob = np.min(log_prob_diffs)
    max_log_prob = np.max(log_prob_diffs)
    normalized_log_probs = 2 * (log_prob_diffs - min_log_prob) / (max_log_prob - min_log_prob) - 1
    return normalized_log_probs

#Calculating the Pearson correlation between model and survey scores

def calculate_correlation(survey_scores, log_prob_diffs):
  if len(survey_scores) != len(log_prob_diffs):
      print(f"Error: Mismatched lengths. Survey scores length: {len(survey_scores)}, Log prob diffs length: {len(log_prob_diffs)}")
      return None, None, None

  normalized_log_probs = normalize_log_prob_diffs(log_prob_diffs)
  try:
        correlation, p_value = pearsonr(survey_scores, normalized_log_probs)
  except Exception as e:
        print("Error during correlation calculation:", e)
        return None, None, None
  return correlation, normalized_log_probs, p_value

### get_batch_last_token_log_prob() function

In [14]:
#Calculating the log probabilities of each of the 2 tokens

def get_batch_last_token_log_prob(lines, model, tokenizer, use_cuda=False, end_with_period=True):
    eos_token = tokenizer.eos_token or tokenizer.sep_token
    if eos_token is None:
        raise ValueError("Neither eos_token nor sep_token is set in the tokenizer.")

    lines = [line + eos_token for line in lines]  #Appending EOS to the end of each line

    tokenizer.pad_token = tokenizer.eos_token

    tok_moral = tokenizer.batch_encode_plus(lines, return_tensors='pt', padding='longest', add_special_tokens=True)
    input_ids = tok_moral['input_ids']
    attention_mask = tok_moral['attention_mask']
    lines_len = torch.sum(attention_mask, dim=1)

    remove_from_end = 2 if end_with_period else 1  #If there is a period remove it as well as the EOS token else remove only the EOS token
    tokens_wanted = lines_len - remove_from_end

    if use_cuda:
        device = next(model.parameters()).device
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        torch.cuda.empty_cache()

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids, return_dict=True)
        logits = outputs.logits
        if use_cuda:
            logits = logits.detach().cpu()

    logits_probs = F.log_softmax(logits, dim=-1)

    batch_indices = torch.arange(input_ids.size(0))
    token_indices = tokens_wanted - 1   #take the index of the token we need. index of 1st element is 0 so we have to do -1
    next_token_indices = input_ids[batch_indices, tokens_wanted].cpu()

    log_probs = logits_probs[batch_indices, token_indices, next_token_indices]

    return log_probs

### get_log_prob_difference() function

In [15]:
#Calculating the average log probability differences between moral and non-moral prompts

def get_log_prob_difference(pairs, moral_index, nonmoral_index, model, tokenizer, use_cuda):
    question_average_lm_score = []
    average_moral_score = []
    average_nonmoral_score = []

    all_prompts = []
    for rating in pairs:
        moral_prompt = rating[moral_index]
        nonmoral_prompt = rating[nonmoral_index]
        all_prompts.append(moral_prompt)
        all_prompts.append(nonmoral_prompt)

    logprobs = get_batch_last_token_log_prob(all_prompts, model, tokenizer, use_cuda)

    for i in range(0, len(logprobs), 2):
        moral_logprob = logprobs[i]
        nonmoral_logprob = logprobs[i + 1]
        lm_score = moral_logprob - nonmoral_logprob

        question_average_lm_score.append(lm_score)
        average_moral_score.append(moral_logprob)
        average_nonmoral_score.append(nonmoral_logprob)

    lm_score = np.mean(question_average_lm_score)
    moral_score = np.mean(average_moral_score)
    nonmoral_score = np.mean(average_nonmoral_score)

    return lm_score, moral_score, nonmoral_score

### compare_token_pair() functions for both datasets

In [16]:
def compare_wvs_token_pairs(model_name, cultures=None, wave=7, excluding_topics=[],
                            excluding_cultures=[], model=None, tokenizer=None, use_cuda=False, prompt_mode='in'):

    if model is None or tokenizer is None:
        tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
        model = transformers.AutoModelForCausalLM.from_pretrained(model_name)
        if use_cuda:
            model = model.cuda()

    wvs_df = get_wvs_df()

    results_all = []

    for culture in tqdm(cultures):
        if culture in excluding_cultures:
            continue
        prompts = wvs_gpt2_prompts_ratings_multiple_tokens(wvs_df, culture, prompt_mode)

        culture_name = culture if culture else 'universal'
        for question, rating_pairs in prompts.items():
            if any(excluded_topic in question for excluded_topic in excluding_topics):
                continue

            lm_score, moral_log_probs, nonmoral_log_probs = get_log_prob_difference(rating_pairs, 0, 1, model, tokenizer, use_cuda)

            if isinstance(moral_log_probs, torch.Tensor):
              moral_log_probs = moral_log_probs.item()
            if isinstance(nonmoral_log_probs, torch.Tensor):
              nonmoral_log_probs = nonmoral_log_probs.item()
            if isinstance(lm_score, torch.Tensor):
              lm_score = lm_score.item()

            wvs_score = rating_pairs[0][2]  #All token pairs have the same wvs_score

            row = {
                'country': culture_name, 'topic': question, 'wvs_score': wvs_score,
                'moral log prob': moral_log_probs, 'non moral log probs': nonmoral_log_probs,
                'log prob difference': lm_score
            }

            results_all.append(row)

    df = pd.DataFrame(results_all)

    survey_scores = df['wvs_score'].values
    log_prob_diffs = df['log prob difference'].values

    correlation, normalized_log_probs, p_value = calculate_correlation(survey_scores, log_prob_diffs)

    df['normalized log prob difference'] = normalized_log_probs

    metrics = {
        'Pearson correlation coefficient': correlation,
        'P value': p_value
    }

    return df, metrics

In [17]:
def compare_pew_token_pairs(model_name, cultures=None, wave=7, excluding_topics=[],
                            excluding_cultures=[], model=None, tokenizer=None, use_cuda=False, prompt_mode='in'):

    if model is None or tokenizer is None:
        tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
        model = transformers.AutoModelForCausalLM.from_pretrained(model_name)
        if use_cuda:
            model = model.cuda()

    pew_df = get_pew_df()

    results_all = []

    for culture in tqdm(cultures):
        if culture in excluding_cultures:
            continue
        prompts = pew_gpt2_prompts_ratings_multiple_tokens(pew_df, culture, prompt_mode)

        culture_name = culture if culture else 'universal'
        for question, rating_pairs in prompts.items():
            if any(excluded_topic in question for excluded_topic in excluding_topics):
                continue

            lm_score, moral_log_probs, nonmoral_log_probs = get_log_prob_difference(rating_pairs, 0, 1, model, tokenizer, use_cuda)

            if isinstance(moral_log_probs, torch.Tensor):
              moral_log_probs = moral_log_probs.item()
            if isinstance(nonmoral_log_probs, torch.Tensor):
              nonmoral_log_probs = nonmoral_log_probs.item()
            if isinstance(lm_score, torch.Tensor):
              lm_score = lm_score.item()

            pew_score = rating_pairs[0][2]  #All token pairs have the same pew_score

            row = {
                'country': culture_name, 'topic': question, 'pew_score': pew_score,
                'moral log prob': moral_log_probs, 'non moral log probs': nonmoral_log_probs,
                'log prob difference': lm_score
            }

            results_all.append(row)

    df = pd.DataFrame(results_all)

    survey_scores = df['pew_score'].values
    log_prob_diffs = df['log prob difference'].values

    correlation, normalized_log_probs, p_value = calculate_correlation(survey_scores, log_prob_diffs)

    df['normalized log prob difference'] = normalized_log_probs

    metrics = {
        'Pearson correlation coefficient': correlation,
        'P value': p_value
    }

    return df, metrics

## Functions only for GPT2 models

In [18]:
#Loading the GPT2 model and the tokenizer

def get_gpt2_model(modelname = 'gpt2', use_cuda = True, device = 'cuda:0'):  #'gpt2','gpt2-medium','gpt2-large','gpt2-xl'
    model = GPT2LMHeadModel.from_pretrained(modelname)
    tokenizer = GPT2Tokenizer.from_pretrained(modelname)

    if use_cuda and torch.cuda.is_available():
        torch.cuda.empty_cache()
        device = torch.device(device)
        model.cuda(device)

    return tokenizer, model

In [19]:
def compare_wvs_token_pairs_gpt2(model_name, cultures=None, wave=7, excluding_topics=[],
                                 excluding_cultures=[], model=None, tokenizer=None, use_cuda=False, prompt_mode='in'):

    if model is None or tokenizer is None:
        tokenizer, model = get_gpt2_model(model_name, use_cuda)

    wvs_df = get_wvs_df()

    gpt2_all = []

    for culture in tqdm(cultures):
        if culture in excluding_cultures:
            continue
        prompts = wvs_gpt2_prompts_ratings_multiple_tokens(wvs_df, culture, prompt_mode)

        culture_name = culture if culture else 'universal'
        for question, rating_pairs in prompts.items():
            if any(excluded_topic in question for excluded_topic in excluding_topics):
                continue

            lm_score, moral_log_probs, nonmoral_log_probs = get_log_prob_difference(rating_pairs, 0, 1, model, tokenizer, use_cuda)

            wvs_score = rating_pairs[0][2]  #All token pairs have the same wvs_score

            row = {
                'country': culture_name, 'topic': question, 'wvs_score': wvs_score,
                'moral log prob': moral_log_probs, 'non moral log probs': nonmoral_log_probs,
                'log prob difference': lm_score
            }

            gpt2_all.append(row)

    df = pd.DataFrame(gpt2_all)

    survey_scores = df['wvs_score'].values
    log_prob_diffs = df['log prob difference'].values

    correlation, normalized_log_probs, p_value = calculate_correlation(survey_scores, log_prob_diffs)
    if correlation is not None:
      print("Correlation calculated successfully")
    else:
      print("Failed to calculate correlation due to data length mismatch")

    df['normalized log prob difference'] = normalized_log_probs

    metrics = {
        'Pearson correlation coefficient': correlation,
        'P value': p_value
    }

    return df, metrics

In [20]:
def compare_pew_token_pairs_gpt2(cultures=None, wave=7, model_name='gpt2',
                                 excluding_topics=[], excluding_cultures=[], model=None, tokenizer=None,
                                 use_cuda=False, prompt_mode='in'):
    if model is None or tokenizer is None:
        tokenizer, model = get_gpt2_model(model_name, use_cuda)

    pew_df = get_pew_df()

    gpt2_all = []

    for culture in tqdm(cultures):
        if culture in excluding_cultures:
            continue
        prompts = pew_gpt2_prompts_ratings_multiple_tokens(pew_df, culture, prompt_mode)

        culture_name = culture if culture else 'universal'
        for question, rating_pairs in prompts.items():
            if any(excluded_topic in question for excluded_topic in excluding_topics):
                continue

            lm_score, moral_log_probs, nonmoral_log_probs = get_log_prob_difference(rating_pairs, 0, 1, model, tokenizer, use_cuda)

            pew_score = rating_pairs[0][2]  #All token pairs have the same pew_score

            row = {
                'country': culture_name, 'topic': question, 'pew_score': pew_score,
                'moral log prob': moral_log_probs, 'non moral log probs': nonmoral_log_probs,
                'log prob difference': lm_score
            }

            gpt2_all.append(row)

    df = pd.DataFrame(gpt2_all)

    survey_scores = df['pew_score'].values
    log_prob_diffs = df['log prob difference'].values

    correlation, normalized_log_probs, p_value = calculate_correlation(survey_scores, log_prob_diffs)
    if correlation is not None:
      print("Correlation calculated successfully")
    else:
      print("Failed to calculate correlation due to data length mismatch")

    df['normalized log prob difference'] = normalized_log_probs

    metrics = {
        'Pearson correlation coefficient': correlation,
        'P value': p_value
    }

    return df, metrics

# 1.Qwen

In [26]:
TOKEN_PAIRS = [ ('always justifiable', 'never justifiable'),
                ('right','wrong'),
                ('morally good','morally bad'),
                ('ethically right', 'ethically wrong'),
                ('ethical','unethical')]

In [ ]:
#Function using Qwen model and WVS datatset

df_Qwen_WVS, metrics_Qwen_WVS = compare_wvs_token_pairs(model_name="Qwen/Qwen2-0.5B",
                                       cultures = COUNTRIES_WVS_W7_ALL, use_cuda=False, prompt_mode='in') #can also use prompt_mode='people'

output_file = 'results_Qwen_WVS.xlsx'
df_Qwen_WVS.to_excel(output_file, index=False)

print(f"Pearson correlation coefficient: {metrics_Qwen_WVS['Pearson correlation coefficient']}")
print(f"P-value: {metrics_Qwen_WVS['P value']}")

In [ ]:
#Function using Qwen model and PEW datatset

df_Qwen_PEW, metrics_Qwen_PEW = compare_pew_token_pairs(model_name="Qwen/Qwen2-0.5B",
                                       cultures = COUNTRIES_PEW_ALL, use_cuda=False, prompt_mode='in') #can also use prompt_mode='people'

output_file = 'results_Qwen_PEW.xlsx'
df_Qwen_PEW.to_excel(output_file, index=False)

print(f"Pearson correlation coefficient: {metrics_Qwen_PEW['Pearson correlation coefficient']}")
print(f"P-value: {metrics_Qwen_PEW['P value']}")

# 2.BLOOM

In [30]:
TOKEN_PAIRS = [ ('always justifiable', 'never justifiable'),
                ('right','wrong'),
                ('morally good','morally bad'),
                ('ethically right', 'ethically wrong'),
                ('ethical','unethical')]

In [ ]:
#Function using BLOOM model and WVS datatset

df_BLOOM_WVS, metrics_BLOOM_WVS = compare_wvs_token_pairs(model_name="bigscience/bloomz-560m",
                                       cultures = COUNTRIES_WVS_W7_ALL, use_cuda=False, prompt_mode='in') #can also use prompt_mode='people'

output_file = 'results_BLOOM_WVS.xlsx'
df_BLOOM_WVS.to_excel(output_file, index=False)

print(f"Pearson correlation coefficient: {metrics_BLOOM_WVS['Pearson correlation coefficient']}")
print(f"P-value: {metrics_BLOOM_WVS['P value']}")

In [ ]:
#Function using BLOOM model and PEW datatset

df_BLOOM_PEW, metrics_BLOOM_PEW = compare_pew_token_pairs(model_name="bigscience/bloomz-560m",
                                       cultures = COUNTRIES_PEW_ALL, use_cuda=False, prompt_mode='in') #can also use prompt_mode='people'

output_file = 'results_BLOOM_PEW.xlsx'
df_BLOOM_PEW.to_excel(output_file, index=False)

print(f"Pearson correlation coefficient: {metrics_BLOOM_PEW['Pearson correlation coefficient']}")
print(f"P-value: {metrics_BLOOM_PEW['P value']}")

# 3.OPT

In [ ]:
TOKEN_PAIRS = [ ('always justifiable', 'never justifiable'),
                ('right','wrong'),
                ('morally good','morally bad'),
                ('ethically right', 'ethically wrong'),
                ('ethical','unethical')]

In [ ]:
#Function using OPT model and WVS datatset

df_OPT_WVS, metrics_OPT_WVS = compare_wvs_token_pairs(model_name="facebook/opt-350m",             #can also use "facebook/opt-350m"
                                       cultures = COUNTRIES_WVS_W7_ALL, use_cuda=False, prompt_mode='in') #can also use prompt_mode='people'

output_file = 'results_OPT_WVS.xlsx'
df_OPT_WVS.to_excel(output_file, index=False)

print(f"\n Pearson correlation coefficient: {metrics_OPT_WVS['Pearson correlation coefficient']}")
print(f"P-value: {metrics_OPT_WVS['P value']}")

In [ ]:
#Function using OPT model and PEW datatset

df_OPT_PEW, metrics_OPT_PEW = compare_pew_token_pairs(model_name="facebook/opt-350m",             #can also use "facebook/opt-350m"
                                       cultures = COUNTRIES_PEW_ALL, use_cuda=False, prompt_mode='in') #can also use prompt_mode='people'

output_file = 'results_OPT_PEW.xlsx'
df_OPT_PEW.to_excel(output_file, index=False)

print(f"\n Pearson correlation coefficient: {metrics_OPT_PEW['Pearson correlation coefficient']}")
print(f"P-value: {metrics_OPT_PEW['P value']}")

# 4.GPT2

In [ ]:
TOKEN_PAIRS = [ ('always justifiable', 'never justifiable'),
                ('right','wrong'),
                ('morally good','morally bad'),
                ('ethically right', 'ethically wrong'),
                ('ethical','unethical')]

In [ ]:
#Function using one of the GPT2 models and WVS datatset

df_GPT2_WVS, metrics_GPT2_WVS = compare_wvs_token_pairs_gpt2(model_name="gpt2",                   #can also use "gpt2-medium" or "gpt2-large"
                                       cultures = COUNTRIES_WVS_W7_ALL, use_cuda=False, prompt_mode='in') #can also use prompt_mode='people'

output_file = 'results_GPT2_base_WVS.xlsx'
df_GPT2_WVS.to_excel(output_file, index=False)

print(f"\n Pearson correlation coefficient: {metrics_GPT2_WVS['Pearson correlation coefficient']}")
print(f"P-value: {metrics_GPT2_WVS['P value']}")

In [ ]:
#Function using one of the GPT2 models and PEW datatset

df_GPT2_PEW, metrics_GPT2_PEW = compare_pew_token_pairs_gpt2(model_name="gpt2",                        #can also use "gpt2-medium" or "gpt2-large"
                                       cultures = COUNTRIES_PEW_ALL, use_cuda=False, prompt_mode='in') #can also use prompt_mode='people'

output_file = 'results_GPT2_base_PEW.xlsx'
df_GPT2_PEW.to_excel(output_file, index=False)

print(f"Pearson correlation coefficient: {metrics_GPT2_PEW['Pearson correlation coefficient']}")
print(f"P-value: {metrics_GPT2_PEW['P value']}")

# (5)Gemma model

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# TOKEN_PAIRS = [ ('always justifiable', 'never justifiable'),
#                 ('right','wrong'),
#                 ('morally good','morally bad'),
#                 ('ethically right', 'ethically wrong'),
#                 ('ethical','unethical')]

TOKEN_PAIRS = [ ('always justifiable', 'never justifiable')]

In [ ]:
#Function using Gemma model and WVS datatset

df_Gemma_WVS, metrics_Gemma_WVS = compare_wvs_token_pairs(model_name="google/gemma-2b-it",
                                       cultures = COUNTRIES_WVS_W7_ALL, use_cuda=False, prompt_mode='in') #can also use prompt_mode='people'

print(f"\n Pearson correlation coefficient: {metrics_Gemma_WVS['Pearson correlation coefficient']}")
print(f"P-value: {metrics_Gemma_WVS['P value']}")

In [ ]:
#Function using Gemma model and WPEWVS datatset

df_Gemma_PEW, metrics_Gemma_PEW = compare_pew_token_pairs(model_name="google/gemma-2b-it",
                                       cultures = COUNTRIES_PEW_ALL, use_cuda=False, prompt_mode='in') #can also use prompt_mode='people'

print(f"Pearson correlation coefficient: {metrics_Gemma_PEW['Pearson correlation coefficient']}")
print(f"P-value: {metrics_Gemma_PEW['P value']}")

# (6)LLaMA model

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
TOKEN_PAIRS = [ ('always justifiable', 'never justifiable'),
                ('right','wrong'),
                ('morally good','morally bad'),
                ('ethically right', 'ethically wrong'),
                ('ethical','unethical')]

# TOKEN_PAIRS = [ ('always justifiable', 'never justifiable')]

In [ ]:
#Function using LLaMA model and WVS datatset

df_LLaMa_WVS, metrics_LLaMa_WVS = compare_wvs_token_pairs(model_name="meta-llama/Meta-Llama-3-8B",
                                       cultures = COUNTRIES_WVS_W7_ALL, use_cuda=False, prompt_mode='in') #can also use prompt_mode='people'

print(f"\n Pearson correlation coefficient: {metrics_LLaMa_WVS['Pearson correlation coefficient']}")
print(f"P-value: {metrics_LLaMa_WVS['P value']}")

In [ ]:
#Function using LLaMA model and PEW datatset

df_LLaMa_PEW, metrics_LLaMa_PEW = compare_pew_token_pairs(model_name="meta-llama/Meta-Llama-3-8B",
                                       cultures = COUNTRIES_PEW_ALL, use_cuda=False, prompt_mode='in') #can also use prompt_mode='people'

print(f"Pearson correlation coefficient: {metrics_LLaMa_PEW['Pearson correlation coefficient']}")
print(f"P-value: {metrics_LLaMa_PEW['P value']}")

# Plots

In [40]:
#WVS datasets

df = df_GPT2_WVS
# df = df_OPT_WVS
# df = df_Qwen_WVS
# df = df_BLOOM_WVS

#PEW datasets

# df = df_GPT2_PEW
# df = df_OPT_PEW
# df = df_Qwen_PEW
# df = df_BLOOM_PEW

In [ ]:
#0. Creating a boxplot to visualize the variations in moral scores between countries across topics in thw WVS dataset
df = df_GPT2_WVS

plt.figure(figsize=(7, 5))  #9,6
sns.boxplot(data=df, x='wvs_score', y='topic', hue='topic', orient='h', palette="husl", legend=False)

plt.xlim(-1, 1)
plt.title('Distribution of WVS Moral Scores by Topic',fontsize=10)
plt.xlabel('')
plt.ylabel('')
plt.tight_layout()
plt.show()

In [ ]:
#1.Creating a boxplot to visualize the variations in moral scores between countries across topics

plt.figure(figsize=(7, 4))  #7, 5 WVS
sns.boxplot(data=df, x='normalized log prob difference', y='topic', hue='topic', orient='h', palette="husl", legend=False)

plt.xlim(-1, 1)
# plt.title('Moral scores inferred from Qwen model for WVS',fontsize=10)  #GPT2 base model  BLOOM-560M  OPT-125M
plt.xlabel('')
plt.ylabel('')
plt.tight_layout()
plt.show()